In [ ]:
from kiwipiepy import Kiwi
from math import log1p
import numpy as np
import konlpy
import nltk
import re

In [ ]:
def preprocess(plaintext, sep,
               morphemeAnalyzer='kiwi', targetMorphs=['NNP','NNG'],removehashtag=True, returnMorph=False,
               returnEnglishMorph=False, eeTagRule={'NNP':'NNP'},
               filterMorphemeAnalyzer='kiwi',filterTargetMorphs=['NNP','NNG','W_HASHTAG'],
               k_1Filter=1.5 ,bFilter=0.75,
               filterThreshold = 3.315):
    
    # 형태소 분석기를 텍스트로 정의 시 해당 형태소 분석기를 할당
    tma = set_morpheme_analyzer(morphemeAnalyzer)
    fma = set_morpheme_analyzer(filterMorphemeAnalyzer)
    
    # 서버에서 "글1 구분자 글2 구분자" 형식의 데이터를 받는다고 가정
    data = plaintext.split(sep)
    # 구분자로 데이터를 받으면 글이 존재하지 않는 마지막 부분을 삭제
    
    if data[-1] in ('',' ','\n',[],['\n'],[' ']):
        data=data[:-1]
    
    if removehashtag == True:
        if '#' in sep:
            sep = sep.replace('#','')
        tdata = plaintext.replace('#',' ').split(sep)
        if tdata[-1] in ('',' ','\n',[],['\n'],[' ']):
            tdata=tdata[:-1]
    else:
        tdata = data*1
            
    postLens = list()
    for post in data:
        postLens.append(len(post))

    flag = (morphemeAnalyzer==filterMorphemeAnalyzer, set(targetMorphs)==set(filterTargetMorphs))
    
    
    ftok = data_tokenize(data,fma,filterTargetMorphs,
                         returnMorph=False,
                         returnEnglishMorph=True,
                         eeTagRule={'W_HASHTAG':'W_HASHTAG'})
    ttok = data_tokenize(data,tma,targetMorphs,
                         returnMorph=returnMorph,
                         returnEnglishMorph=returnEnglishMorph, eeTagRule=eeTagRule)
    
    filterScores = BM25(ftok, postLens, k_1=k_1Filter, b=bFilter)
    
    spamCount=0
    for idx in range(len(filterScores)):
        if filterScores[idx] < filterThreshold:
            spamCount+=1
            ttok.pop(idx-spamCount)
    print("%s 개의 데이터가 삭제되었습니다."%spamCount)
    
    return ttok

In [ ]:
def set_morpheme_analyzer(maText):
    if maText in ['kiwi','Kiwi','KIWI','키위']:
        return Kiwi()
    elif maText in ['Hannanum', 'hannanum', 'HANNANUM','한나눔']:
        return konlpy.tag.Hannanum()
    elif maText in ['Komoran','KOMORAN','komoran','코모란']:
        return konlpy.tag.Komoran()
    elif maText in ['Kkma','KKMA','kkma','꼬꼬마']:
        return konlpy.tag.Kkma()
    elif maText in ['Okt','OKT','okt','오픈코리안텍스트','트위터']:
        return konlpy.tag.Okt()
    elif maText in ['Mecab','mecab','MECAB','미캐브']:
        return konlpy.tag.Mecab()
    else:
        raise Exception('No such morpheme analyzer\nSupported morpheme analyzers are Kiwi, KoNLPy(Hannanum, Komoran, Kkma, Okt, Mecab)')

In [ ]:
def data_tokenize(data,morphemeAnalyzer,targetMorph,
                  returnMorph=False,
                  returnEnglishMorph=False,
                  eeTagRule={'NNP':'NNP'}):
    
    returnData = list()
    
    maDir = dir(morphemeAnalyzer)
    if 'pos' in maDir:
        for post in data:
            selected = list()
            if returnEnglishMorph==True:
                eeTags, post = emoji_english_preprocess(post, tagRule=eeTagRule, returnMorph=returnMorph)
                selected += eeTags
            
            for tok in morphemeAnalyzer.pos(post):
                if tok[1] in targetMorph:
                    if returnMorph==True:
                        selected.append((tok[0],tok[1]))
                    else:
                        selected.append(tok[0])
            returnData.append(selected)
    
    elif 'tokenize' in maDir:
        for post in data:
            selected = list()
            if returnEnglishMorph==True:
                eeTags, post = emoji_english_preprocess(post, tagRule=eeTagRule, returnMorph=returnMorph)
                selected += eeTags
            
            for tok in morphemeAnalyzer.tokenize(post):
                if tok.tag in targetMorph:
                    if returnMorph==True:
                        selected.append((tok.form,tok.tag))
                    else:
                        selected.append(tok.form)
            returnData.append(selected)
    else:
        raise Exception('Not supported morpheme analyzer instance')
    return returnData

In [ ]:
def BM25(data, postLens, k_1=1.5, b=0.75):
    avgPostLen = np.mean(postLens)
    
    N = len(data)
    
    n = dict()
    for post in data:
        uniqueToks = set(post)
        for tok in uniqueToks:
            try:
                n[tok]+=1
            except:
                n[tok] = 1
    
    IDF = dict()
    for tok in n.keys():
        IDF[tok] = log1p((N-n[tok]+0.5)/(n[tok]+0.5))


    filterScores = list()

    for postidx, post in enumerate(data):
        postScore = 0
        for tok in post:
            tokCount = post.count(tok)
            postScore += (IDF[tok] * (
                (tokCount*(k_1+1))/(
                    tokCount+(k_1*(1-b+(b*(postLens[postidx]/avgPostLen)))))))
        try:
            filterScores.append((postScore/len(post)))
        except:
            filterScores.append(0)

    return filterScores

In [ ]:
def emoji_english_preprocess(post, tagRule={'NNP':'NNP'}, returnMorph=True):
    returnData = list()
    
    emojis = re.findall(':[_A-Za-z]+:',post)
    for emoji in set(emojis):
        emojiCounts = post.count(emoji)
        post = post.replace(emoji,'')
        returnData+=([(emoji,'EMJ')]*emojiCounts)
        
    hashTags = re.findall('#[_A-Za-z]',post)
    for hashTag in set(hashTags):
        hTagCounts = post.count(hashTag)
        post = post.replace(hashTag,'')
        returnData+=([(hashTag,'W_HASHTAG')]*hTagCounts)

    
    engChunks = re.findall('[A-Za-z]+[\' ]?[A-Za-z]+',post)
    for engChunk in set(engChunks):
        chunkCounts = post.count(engChunk)
        post = post.replace(engChunk,'')

        targetToken = list()
        for token in nltk.pos_tag(nltk.word_tokenize(engChunk)):
            if token[1] in tagRule: 
                targetToken.append((token[0],token[1]))
        returnData+=(targetToken*chunkCounts)

    filterData = list()
    for token in returnData:
        if token[1] in tagRule:
            if returnMorph==True:
                filterData.append((token[0],tagRule[token[1]]))
            else:
                filterData.append(token[0])
        
    return filterData, post